In [ ]:
#!pip install tensorflow-gpu==2.3

In [1]:
from google.colab import files
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,Embedding,Flatten
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
uploaded=files.upload()

Saving prepared_data.csv to prepared_data.csv


In [3]:
data=pd.read_csv('prepared_data.csv')
data.head()

,SMS_TEXT,ENGLISH_TEXT
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [4]:
data['SMS_LEN'] = data['SMS_TEXT'].str.split().apply(len)
data = data[data['SMS_LEN'] < 39]



data['english_len'] = data['ENGLISH_TEXT'].str.split().apply(len)
data = data[data['english_len'] < 40]

data['ENGLISH_INPUT'] = '<start> ' + data['ENGLISH_TEXT'].astype(str)
data['ENGLISH_OUTPUT'] = data['ENGLISH_TEXT'].astype(str) + ' <end>'

data = data.drop(['ENGLISH_TEXT','SMS_LEN','english_len'], axis=1)
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer

print(data.shape)
data.iloc[0]['ENGLISH_INPUT']=str(data.iloc[0]['ENGLISH_INPUT'])+' <end>'
data.iloc[0]['ENGLISH_OUTPUT']=str(data.iloc[0]['ENGLISH_OUTPUT'])+' <end>'

(1988, 3)


In [5]:
from sklearn.model_selection import train_test_split
train_data,test_data= train_test_split(data,test_size=0.01, random_state=42)
print(train_data.shape)
print(test_data.shape)

(1968, 3)
(20, 3)


In [6]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n',char_level=False,lower=True,oov_token=True)
print("SMS_TEXT")
tokenizer.fit_on_texts(train_data['SMS_TEXT'].values)
print("English text")
tokenizer_e = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n',char_level=False,lower=True,oov_token=True)
tokenizer_e.fit_on_texts(train_data['ENGLISH_INPUT'].values)

SMS_TEXT
English text


In [9]:
encoder_vocabluary=tokenizer.word_index
decoder_vocabluary=tokenizer_e.word_index
print(len(encoder_vocabluary))
print(len(decoder_vocabluary))


3701
3039


In [10]:
print(encoder_vocabluary)

{True: 1, 'u': 2, 'i': 3, 'to': 4, 'me': 5, 'ü': 6, 'at': 7, 'go': 8, 'my': 9, 'so': 10, 'a': 11, 'can': 12, 'you': 13, 'the': 14, 'not': 15, 'haha': 16, 'e': 17, 'hey': 18, 'got': 19, 'in': 20, 'for': 21, 'is': 22, 'ur': 23, 'lor': 24, "i'm": 25, 'on': 26, 'ok': 27, 'now': 28, 'time': 29, 'but': 30, 'no': 31, 'then': 32, 'it': 33, '2': 34, 'wat': 35, 'n': 36, 'we': 37, 'of': 38, 'or': 39, 'how': 40, 'r': 41, '4': 42, 'if': 43, 'ah': 44, 'la': 45, 'hi': 46, 'dun': 47, 'one': 48, 'den': 49, 'will': 50, 'meet': 51, 'do': 52, 'wan': 53, 'have': 54, 'all': 55, 'liao': 56, 'out': 57, 'be': 58, 'and': 59, 'up': 60, 'tmr': 61, 'going': 62, 'still': 63, 'are': 64, 'leh': 65, 'call': 66, 'where': 67, 'k': 68, 'ask': 69, 'ya': 70, "i'll": 71, 'too': 72, 'oh': 73, 'there': 74, 'get': 75, 'come': 76, 'when': 77, 'like': 78, 'goin': 79, 'home': 80, 'im': 81, 'later': 82, 'today': 83, 'b': 84, 'take': 85, 'your': 86, 'she': 87, 'back': 88, 'think': 89, 'oredi': 90, 'juz': 91, 'just': 92, 'wanna': 93

In [11]:
print(decoder_vocabluary)

{True: 1, '<start>': 2, 'you': 3, 'i': 4, 'to': 5, 'the': 6, 'are': 7, 'is': 8, 'and': 9, 'for': 10, 'at': 11, 'me': 12, 'a': 13, 'so': 14, 'my': 15, 'can': 16, 'then': 17, 'your': 18, 'go': 19, "i'm": 20, 'it': 21, 'not': 22, 'have': 23, "don't": 24, 'ok': 25, 'want': 26, 'haha': 27, 'what': 28, 'in': 29, 'do': 30, 'hey': 31, 'going': 32, 'be': 33, 'of': 34, 'now': 35, 'on': 36, 'that': 37, 'or': 38, 'will': 39, 'just': 40, 'but': 41, 'how': 42, 'we': 43, 'got': 44, 'know': 45, 'time': 46, 'with': 47, 'see': 48, 'am': 49, 'already': 50, 'no': 51, 'tomorrow': 52, 'if': 53, 'hi': 54, 'think': 55, 'yes': 56, 'meet': 57, "it's": 58, 'good': 59, 'all': 60, 'out': 61, 'there': 62, 'still': 63, 'okay': 64, 'because': 65, 'where': 66, 'come': 67, 'call': 68, 'up': 69, 'one': 70, 'also': 71, 'too': 72, 'like': 73, 'when': 74, "i'll": 75, 'get': 76, 'this': 77, 'from': 78, 'about': 79, 'oh': 80, 'home': 81, 'today': 82, 'back': 83, 'need': 84, 'very': 85, 'right': 86, 'why': 87, 'later': 88, 't

In [ ]:
from joblib import dump, load
decoder_embedding=load('/content/decoder_embedding2.joblib')
print(decoder_embedding.shape)
encoder_embedding=load('/content/encoder_embedding1.joblib')
print(encoder_embedding.shape)


(3040, 300)
(3702, 300)


In [ ]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):
        self.inp_vocab_size=inp_vocab_size
        self.embedding_size=embedding_size
        self.lstm_size=lstm_size
        self.input_length=input_length

        #Initialize Embedding layer

        #Intialize Encoder LSTM layer
        super().__init__()
        self.Embedding_Layer=tf.keras.layers.Embedding(input_dim=self.inp_vocab_size,output_dim=self.embedding_size,input_length=input_length,mask_zero=True,name="Encoder_Embedding",weights=[encoder_embedding])
        self.LSTM_Layer=tf.keras.layers.LSTM(self.lstm_size,return_state=True,return_sequences=True,name="Encoder_LSTM")

    def call(self,input_sequence,states):
      '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
      '''
      input_embedd=self.Embedding_Layer(input_sequence)
      self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.LSTM_Layer(input_embedd)
     
      return self.lstm_output,self.lstm_state_h,self.lstm_state_c

    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      intial_hidden_state=np.zeros((batch_size,self.lstm_size))
      intial_cell_state=np.zeros((batch_size,self.lstm_size))
      return intial_hidden_state,intial_cell_state

In [ ]:
class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,scoring_function, att_units):
    self.scoring_function=scoring_function
    self.att_units=att_units 
    super().__init__()
 
    # Please go through the reference notebook and research paper to complete the scoring functions
 
    if self.scoring_function=='dot':
      # Intialize variables needed for Dot score function here
      pass
 
      
      
    if scoring_function == 'general':
      # Intialize variables needed for General score function here
      self.W=tf.keras.layers.Dense(att_units)
    elif scoring_function == 'concat':
      # Intialize variables needed for Concat score function here
      self.W1 = tf.keras.layers.Dense(att_units)
      self.W2 = tf.keras.layers.Dense(att_units)
      self.V = tf.keras.layers.Dense(1)
  
  
  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    
    if self.scoring_function == 'dot':
        # Implement Dot score function here
        
        
        scoring=tf.matmul(encoder_output,tf.expand_dims(decoder_hidden_state,axis=-1))
        
        attention_weight=tf.nn.softmax(scoring,axis=1)
        
        context_vector=attention_weight*encoder_output
        context_vector=tf.reduce_sum(context_vector,axis=1)
        return context_vector,attention_weight
              
        
        
        
 
        
    elif self.scoring_function == 'general':
        # Implement General score function here
        
        score=tf.matmul(self.W(encoder_output),tf.expand_dims(decoder_hidden_state,axis=-1))
        attention_weight=tf.nn.softmax(score,axis=1)
        context_vector=attention_weight*encoder_output
        context_vector=tf.reduce_sum(context_vector,axis=1)
        return context_vector,attention_weight
    
    elif self.scoring_function == 'concat':
        # Implement General score function here
        score = self.V(tf.nn.tanh(self.W1(encoder_output) + self.W2(tf.expand_dims(decoder_hidden_state,axis=1))))
        attention_weight=tf.nn.softmax(score,axis=1)
        context_vector=attention_weight*encoder_output
        context_vector=tf.reduce_sum(context_vector,axis=1)
        return context_vector,attention_weight

In [ ]:
class OneStepDecoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):

      # Initialize decoder embedding layer, LSTM and any other objects needed
      self.tar_vocab_size=tar_vocab_size
      self.embedding_dim=embedding_dim
      self.input_length=input_length
      self.dec_units=dec_units
      self.score_fun=score_fun
      self.att_units=att_units
      super().__init__()
      self.DEmbedding=tf.keras.layers.Embedding(input_dim=self.tar_vocab_size,output_dim=self.embedding_dim,input_length=self.input_length,mask_zero=True,name="Decoder_Embedding", trainable=False,weights=[decoder_embedding])
      self.DLSTM_Layer=tf.keras.layers.LSTM(self.dec_units,return_state=True,return_sequences=True,name="Encoder_LSTM")
      self.full_connected=tf.keras.layers.Dense(self.tar_vocab_size)
      self.attention=Attention(self.score_fun,self.att_units)




  def call(self,input_to_decoder, encoder_output, state_h,state_c):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    #step-A
    output=self.DEmbedding(input_to_decoder)
    #step-B
   
    context_vector,attention_weights=self.attention(state_h,encoder_output)
    #step-C
    concat=tf.concat([tf.expand_dims(context_vector,1),output],axis=-1)
    #step-D
    decoder_output,hidden_states,cell_states=self.DLSTM_Layer(concat,initial_state=[state_h,state_c])
    decoder_output=tf.reshape(decoder_output,(-1,decoder_output.shape[2]))
    output=self.full_connected(decoder_output)
    return output,hidden_states,cell_states,attention_weights,context_vector

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      self.out_vocab_size=out_vocab_size
      self.embedding_dim=embedding_dim
      self.input_length=input_length
      self.dec_units=dec_units
      self.score_fun=score_fun
      self.att_units=att_units
      super().__init__()
      self.onestepdecoder=OneStepDecoder(self.out_vocab_size, self.embedding_dim, self.input_length, self.dec_units ,self.score_fun ,self.att_units)


     
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook
        
        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
        
        all_outputs=tf.TensorArray(tf.float32,size=tf.shape(input_to_decoder)[1],name="output_arrays")
      
        
        for timestep in range(tf.shape(input_to_decoder)[1]):
            output,decoder_hidden_state,decoder_cell_state,_,_=self.onestepdecoder(input_to_decoder[:,timestep:timestep+1],encoder_output,decoder_hidden_state,decoder_cell_state)
            all_outputs=all_outputs.write(timestep,output)
            
        all_outputs=tf.transpose(all_outputs.stack(),[1,0,2])
        return all_outputs


In [ ]:

class encoder_decoder(tf.keras.Model):
    def __init__(self,embedding_size,lstm_size,input_length,decoder_input_length,dec_units ,score_fun ,att_units, batch_size):
        #Intialize objects from encoder decoder
        super().__init__()
        self.encoder=Encoder(len(encoder_vocabluary)+1,embedding_size,lstm_size,input_length)
        self.decoder=Decoder(len(decoder_vocabluary)+1,embedding_size, decoder_input_length, dec_units ,score_fun ,att_units)
        self.batch_size= batch_size
    def call(self,data):
        #Intialize encoder states, Pass the encoder_sequence to the embedding layer
        # Decoder initial states are encoder final states, Initialize it accordingly
        # Pass the decoder sequence,encoder_output,decoder states to Decoder
        # return the decoder output
        input_,target_sentences = data[0], data[1]
        initial_state= self.encoder.initialize_states(self.batch_size)
        encoder_output,state_h,state_c=self.encoder(input_,initial_state)
        output=self.decoder(target_sentences,encoder_output, state_h, state_c)
        return output

In [ ]:
def custom_lossfunction(targets,logits):

  # Custom loss function that will not consider the loss for padded zeros.
  # Refer https://www.tensorflow.org/tutorials/text/nmt_with_attention#define_the_optimizer_and_the_loss_function
   
   loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
   mask = tf.math.logical_not(tf.math.equal(targets, 0))
   loss_ = loss_object(targets, logits)

   mask = tf.cast(mask, dtype=loss_.dtype)
   loss_ *= mask

   return tf.reduce_mean(loss_)

In [ ]:
class Dataset:
    def __init__(self, data, tknizer_ita, tknizer_eng, max_len,max_len1):
        self.encoder_inps = data['SMS_TEXT'].values
        self.decoder_inps = data['ENGLISH_INPUT'].values
        self.decoder_outs = data['ENGLISH_OUTPUT'].values
        self.tknizer_eng = tknizer_eng
        self.tknizer_ita = tknizer_ita
        self.max_len = max_len
        self.max_len1=max_len1
 
    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_ita.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_eng.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_eng.texts_to_sequences([self.decoder_outs[i]])
 
        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len1, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len1, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq
 
    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)
 
    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))
 
 
    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
 
        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])
 
    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size
 
    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [ ]:
tf.keras.backend.clear_session()
train_dataset = Dataset(train_data, tokenizer, tokenizer_e, 38,39)
test_dataset  = Dataset(test_data, tokenizer, tokenizer_e, 38,39)
train_dataloader = Dataloder(train_dataset, batch_size=64)
test_dataloader = Dataloder(test_dataset, batch_size=20)
model2=encoder_decoder(300,256,38,39,256,'concat',256,64)
optimizer = tf.keras.optimizers.Adam(0.01)
model2.compile(optimizer=optimizer,loss=custom_lossfunction)
train_steps=train_data.shape[0]//64
valid_steps=test_data.shape[0]//20
model2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=40, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
30/30 [==============================] - 30s 710ms/step - loss: 2.5480 - val_loss: 2.3325
Epoch 2/40
30/30 [==============================] - 20s 672ms/step - loss: 2.2365 - val_loss: 2.2937
Epoch 3/40
30/30 [==============================] - 19s 623ms/step - loss: 2.1629 - val_loss: 2.2443
Epoch 4/40
30/30 [==============================] - 19s 645ms/step - loss: 2.0915 - val_loss: 2.1800
Epoch 5/40
30/30 [==============================] - 19s 631ms/step - loss: 1.9005 - val_loss: 2.1036
Epoch 6/40
30/30 [==============================] - 19s 624ms/step - loss: 1.8547 - val_loss: 2.0614
Epoch 7/40
30/30 [==============================] - 19s 635ms/step - loss: 1.7162 - val_loss: 1.9904
Epoch 8/40
30/30 [==============================] - 19s 635ms/step - loss: 1.5961 - val_loss: 1.9626
Epoch 9/40
30/30 [==============================] - 19s 640ms/step - loss: 1.4634 - val_loss: 1.9375
Epoch 10/40
30/30 [==============================] - 19s 622ms/step - loss: 1.3497 - val_lo

KeyboardInterrupt: ignored

In [ ]:
tf.keras.backend.clear_session()
train_dataset = Dataset(train_data, tokenizer, tokenizer_e, 38,39)
test_dataset  = Dataset(test_data, tokenizer, tokenizer_e, 38,39)
train_dataloader = Dataloder(train_dataset, batch_size=64)
test_dataloader = Dataloder(test_dataset, batch_size=20)
model2=encoder_decoder(300,256,38,39,256,'concat',256,64)
optimizer = tf.keras.optimizers.Adam()
model2.compile(optimizer=optimizer,loss=custom_lossfunction)
train_steps=train_data.shape[0]//64
valid_steps=test_data.shape[0]//20
model2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=40, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
30/30 [==============================] - 28s 678ms/step - loss: 2.7253 - val_loss: 2.3231
Epoch 2/40
30/30 [==============================] - 18s 584ms/step - loss: 2.3340 - val_loss: 2.3000
Epoch 3/40
30/30 [==============================] - 17s 582ms/step - loss: 2.2256 - val_loss: 2.2845
Epoch 4/40
30/30 [==============================] - 18s 600ms/step - loss: 2.2436 - val_loss: 2.2762
Epoch 5/40
30/30 [==============================] - 18s 598ms/step - loss: 2.2103 - val_loss: 2.2775
Epoch 6/40
30/30 [==============================] - 17s 571ms/step - loss: 2.1695 - val_loss: 2.2658
Epoch 7/40
30/30 [==============================] - 17s 578ms/step - loss: 2.1364 - val_loss: 2.2475
Epoch 8/40
30/30 [==============================] - 18s 585ms/step - loss: 2.1521 - val_loss: 2.2330
Epoch 9/40
30/30 [==============================] - 18s 589ms/step - loss: 2.1376 - val_loss: 2.2171
Epoch 10/40
30/30 [==============================] - 17s 577ms/step - loss: 2.1100 - val_lo

KeyboardInterrupt: ignored

In [ ]:
tf.keras.backend.clear_session()
train_dataset = Dataset(train_data, tokenizer, tokenizer_e, 38,39)
test_dataset  = Dataset(test_data, tokenizer, tokenizer_e, 38,39)
train_dataloader = Dataloder(train_dataset, batch_size=64)
test_dataloader = Dataloder(test_dataset, batch_size=20)
model2=encoder_decoder(300,100,38,39,100,'concat',100,64)
optimizer = tf.keras.optimizers.Adam()
model2.compile(optimizer=optimizer,loss=custom_lossfunction)
train_steps=train_data.shape[0]//64
valid_steps=test_data.shape[0]//20
model2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=40, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
30/30 [==============================] - 28s 671ms/step - loss: 2.9386 - val_loss: 2.3748
Epoch 2/40
30/30 [==============================] - 19s 621ms/step - loss: 2.2915 - val_loss: 2.3211
Epoch 3/40
30/30 [==============================] - 18s 604ms/step - loss: 2.2410 - val_loss: 2.3035
Epoch 4/40
30/30 [==============================] - 18s 600ms/step - loss: 2.2652 - val_loss: 2.2948
Epoch 5/40
30/30 [==============================] - 18s 613ms/step - loss: 2.2667 - val_loss: 2.2936
Epoch 6/40
30/30 [==============================] - 18s 604ms/step - loss: 2.2286 - val_loss: 2.2865
Epoch 7/40
30/30 [==============================] - 19s 614ms/step - loss: 2.2176 - val_loss: 2.2844
Epoch 8/40
30/30 [==============================] - 19s 646ms/step - loss: 2.1990 - val_loss: 2.2759
Epoch 9/40
30/30 [==============================] - 19s 642ms/step - loss: 2.1585 - val_loss: 2.2676
Epoch 10/40
30/30 [==============================] - 19s 613ms/step - loss: 2.1800 - val_lo

In [ ]:
model2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=100, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
30/30 [==============================] - 18s 594ms/step - loss: 1.6409 - val_loss: 1.9708
Epoch 2/100
30/30 [==============================] - 19s 620ms/step - loss: 1.6241 - val_loss: 1.9662
Epoch 3/100
30/30 [==============================] - 18s 601ms/step - loss: 1.6067 - val_loss: 1.9604
Epoch 4/100
30/30 [==============================] - 18s 597ms/step - loss: 1.5891 - val_loss: 1.9544
Epoch 5/100
30/30 [==============================] - 17s 581ms/step - loss: 1.5730 - val_loss: 1.9537
Epoch 6/100
30/30 [==============================] - 18s 591ms/step - loss: 1.5600 - val_loss: 1.9370
Epoch 7/100
30/30 [==============================] - 19s 619ms/step - loss: 1.5440 - val_loss: 1.9534
Epoch 8/100
13/30 [============>.................] - ETA: 10s - loss: 1.5152

KeyboardInterrupt: ignored

In [ ]:
tf.keras.backend.clear_session()
train_dataset = Dataset(train_data, tokenizer, tokenizer_e, 38,39)
test_dataset  = Dataset(test_data, tokenizer, tokenizer_e, 38,39)
train_dataloader = Dataloder(train_dataset, batch_size=64)
test_dataloader = Dataloder(test_dataset, batch_size=20)
model2=encoder_decoder(300,100,38,39,100,'concat',100,64)
optimizer = tf.keras.optimizers.Adam()
model2.compile(optimizer=optimizer,loss=custom_lossfunction)
train_steps=train_data.shape[0]//64
valid_steps=test_data.shape[0]//20
model2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=40, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
30/30 [==============================] - 18s 419ms/step - loss: 3.0607 - val_loss: 2.4684
Epoch 2/40
30/30 [==============================] - 11s 373ms/step - loss: 2.3780 - val_loss: 2.4156
Epoch 3/40
30/30 [==============================] - 11s 383ms/step - loss: 2.4028 - val_loss: 2.3981
Epoch 4/40
30/30 [==============================] - 11s 377ms/step - loss: 2.3232 - val_loss: 2.3855
Epoch 5/40
30/30 [==============================] - 11s 371ms/step - loss: 2.3508 - val_loss: 2.3808
Epoch 6/40
30/30 [==============================] - 11s 371ms/step - loss: 2.2957 - val_loss: 2.3694
Epoch 7/40
30/30 [==============================] - 11s 372ms/step - loss: 2.2575 - val_loss: 2.3678
Epoch 8/40
30/30 [==============================] - 11s 369ms/step - loss: 2.3214 - val_loss: 2.3628
Epoch 9/40
30/30 [==============================] - 11s 375ms/step - loss: 2.2449 - val_loss: 2.3546
Epoch 10/40
30/30 [==============================] - 11s 374ms/step - loss: 2.2361 - val_lo

In [ ]:
tf.keras.backend.clear_session()
train_dataset = Dataset(train_data, tokenizer, tokenizer_e, 38,39)
test_dataset  = Dataset(test_data, tokenizer, tokenizer_e, 38,39)
train_dataloader = Dataloder(train_dataset, batch_size=64)
test_dataloader = Dataloder(test_dataset, batch_size=20)
model2=encoder_decoder(300,100,38,39,100,'concat',100,64)
optimizer = tf.keras.optimizers.Adam(0.0001)
model2.compile(optimizer=optimizer,loss=custom_lossfunction)
train_steps=train_data.shape[0]//64
valid_steps=test_data.shape[0]//20
model2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=40, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
30/30 [==============================] - 18s 418ms/step - loss: 3.1148 - val_loss: 3.2005
Epoch 2/40
30/30 [==============================] - 11s 374ms/step - loss: 3.1212 - val_loss: 3.1161
Epoch 3/40
30/30 [==============================] - 11s 372ms/step - loss: 2.9978 - val_loss: 2.9095
Epoch 4/40
30/30 [==============================] - 11s 376ms/step - loss: 2.8306 - val_loss: 2.7309
Epoch 5/40
30/30 [==============================] - 12s 385ms/step - loss: 2.6478 - val_loss: 2.6179
Epoch 6/40
30/30 [==============================] - 12s 384ms/step - loss: 2.5707 - val_loss: 2.5486
Epoch 7/40
30/30 [==============================] - 11s 370ms/step - loss: 2.4894 - val_loss: 2.5037
Epoch 8/40
30/30 [==============================] - 11s 374ms/step - loss: 2.4384 - val_loss: 2.4745
Epoch 9/40
30/30 [==============================] - 11s 372ms/step - loss: 2.4195 - val_loss: 2.4557
Epoch 10/40
30/30 [==============================] - 11s 371ms/step - loss: 2.3831 - val_lo

In [ ]:
model2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=100, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
30/30 [==============================] - 11s 366ms/step - loss: 2.3231 - val_loss: 2.3952
Epoch 2/100
30/30 [==============================] - 11s 369ms/step - loss: 2.3215 - val_loss: 2.3937
Epoch 3/100
30/30 [==============================] - 11s 366ms/step - loss: 2.3199 - val_loss: 2.3931
Epoch 4/100
30/30 [==============================] - 11s 367ms/step - loss: 2.3178 - val_loss: 2.3921
Epoch 5/100
30/30 [==============================] - 11s 384ms/step - loss: 2.3162 - val_loss: 2.3907
Epoch 6/100
30/30 [==============================] - 11s 369ms/step - loss: 2.3145 - val_loss: 2.3901
Epoch 7/100
30/30 [==============================] - 11s 379ms/step - loss: 2.3127 - val_loss: 2.3891
Epoch 8/100
30/30 [==============================] - 11s 368ms/step - loss: 2.3111 - val_loss: 2.3884
Epoch 9/100
30/30 [==============================] - 11s 369ms/step - loss: 2.3093 - val_loss: 2.3875
Epoch 10/100
30/30 [==============================] - 11s 367ms/step - loss: 2.307

In [ ]:
tf.keras.backend.clear_session()
train_dataset = Dataset(train_data, tokenizer, tokenizer_e, 38,39)
test_dataset  = Dataset(test_data, tokenizer, tokenizer_e, 38,39)
train_dataloader = Dataloder(train_dataset, batch_size=64)
test_dataloader = Dataloder(test_dataset, batch_size=20)
model2=encoder_decoder(300,100,38,39,100,'concat',100,64)
optimizer = tf.keras.optimizers.Adam(0.1)
model2.compile(optimizer=optimizer,loss=custom_lossfunction)
train_steps=train_data.shape[0]//64
valid_steps=test_data.shape[0]//20
model2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=40, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
30/30 [==============================] - 18s 412ms/step - loss: 3.9617 - val_loss: 3.4156
Epoch 2/40
30/30 [==============================] - 11s 372ms/step - loss: 2.8160 - val_loss: 2.5687
Epoch 3/40
30/30 [==============================] - 11s 378ms/step - loss: 2.4718 - val_loss: 2.4137
Epoch 4/40
30/30 [==============================] - 11s 371ms/step - loss: 2.2830 - val_loss: 2.3402
Epoch 5/40
30/30 [==============================] - 11s 372ms/step - loss: 2.2488 - val_loss: 2.2761
Epoch 6/40
30/30 [==============================] - 11s 371ms/step - loss: 2.1767 - val_loss: 2.2434
Epoch 7/40
30/30 [==============================] - 11s 374ms/step - loss: 2.0705 - val_loss: 2.2030
Epoch 8/40
30/30 [==============================] - 11s 369ms/step - loss: 2.0143 - val_loss: 2.1875
Epoch 9/40
30/30 [==============================] - 11s 375ms/step - loss: 1.9741 - val_loss: 2.1875
Epoch 10/40
30/30 [==============================] - 11s 374ms/step - loss: 1.8782 - val_lo

KeyboardInterrupt: ignored

In [ ]:
tf.keras.backend.clear_session()
train_dataset = Dataset(train_data, tokenizer, tokenizer_e, 38,39)
test_dataset  = Dataset(test_data, tokenizer, tokenizer_e, 38,39)
train_dataloader = Dataloder(train_dataset, batch_size=64)
test_dataloader = Dataloder(test_dataset, batch_size=20)
model2=encoder_decoder(300,100,38,39,100,'concat',100,64)
optimizer = tf.keras.optimizers.Adam(0.01)
model2.compile(optimizer=optimizer,loss=custom_lossfunction)
train_steps=train_data.shape[0]//64
valid_steps=test_data.shape[0]//20
model2.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=40, validation_data=test_dataloader, validation_steps=valid_steps)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
30/30 [==============================] - 29s 684ms/step - loss: 2.6496 - val_loss: 2.3106
Epoch 2/40
30/30 [==============================] - 18s 598ms/step - loss: 2.2993 - val_loss: 2.2977
Epoch 3/40
30/30 [==============================] - 20s 665ms/step - loss: 2.2164 - val_loss: 2.2751
Epoch 4/40
30/30 [==============================] - 18s 608ms/step - loss: 2.1375 - val_loss: 2.2674
Epoch 5/40
30/30 [==============================] - 18s 610ms/step - loss: 2.1264 - val_loss: 2.2608
Epoch 6/40
30/30 [==============================] - 19s 622ms/step - loss: 2.0774 - val_loss: 2.2206
Epoch 7/40
30/30 [==============================] - 18s 605ms/step - loss: 2.0220 - val_loss: 2.1793
Epoch 8/40
30/30 [==============================] - 19s 646ms/step - loss: 1.9573 - val_loss: 2.1448
Epoch 9/40
30/30 [==============================] - 19s 638ms/step - loss: 1.9120 - val_loss: 2.1012
Epoch 10/40
30/30 [==============================] - 19s 615ms/step - loss: 1.8100 - val_lo

KeyboardInterrupt: ignored

In [ ]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to onestepdecoder.
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predictions, input_states, attention_weights = model.layers[1].onestepdecoder(input_to_decoder, encoder_output, input_states)
         Save the attention weights
         And get the word using the tokenizer(word index) and then store it in a string.
  E. Call plot_attention(#params)
  F. Return the predicted sentence

  '''


  input_sequence=tokenizer.texts_to_sequences([input_sentence])
  

  inputs=pad_sequences(input_sequence,maxlen=38,padding='post')
  inputs=tf.convert_to_tensor(inputs)
  result=''
  units=256
  hidden=[tf.zeros((1,units))]
  encoder_output,hidden_state,cell_state=model2.layers[0](inputs,hidden)
  dec_hidden=hidden_state
  dec_input=tf.expand_dims([tokenizer_e.word_index['<start>']],0)
  for t in range(39):
      predictions,dec_hidden,cell_state,attention_weights,context_vector=model2.layers[1].onestepdecoder(dec_input,encoder_output,dec_hidden,cell_state)

      predicted_id=tf.argmax(predictions[0]).numpy()
      result+=tokenizer_e.index_word[predicted_id]
      if tokenizer_e.word_index['<end>']==predicted_id:
          return result
      dec_input= tf.expand_dims([predicted_id],0)
  return result